# Lineare Regression für Umsatzdaten

In diesem Notebook führen wir eine lineare Regression mit den gekürzten Umsatzdaten durch.

In [ ]:
# Benötigte Bibliotheken importieren
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Daten einlesen
df = pd.read_csv('umsatzdaten_gekuerzt.csv')

# Ersten Überblick über die Daten verschaffen
print("Datensatzgröße:", df.shape)
print("\nSpalten im Datensatz:")
print(df.columns.tolist())
print("\nErste Zeilen des Datensatzes:")
df.head()

In [ ]:
# Daten vorbereiten für die lineare Regression
# Hier müssen wir die Feature-Spalten (X) und die Ziel-Spalte (y) definieren
# Wir nehmen an, dass wir den Umsatz vorhersagen wollen

# Features und Zielwerte werden nach Analyse der Datensatzstruktur definiert
# X = df[['feature1', 'feature2', ...]]
# y = df['target']

# Daten in Trainings- und Testset aufteilen
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Lineares Regressionsmodell erstellen und trainieren
# model = LinearRegression()
# model.fit(X_train, y_train)

# Vorhersagen machen
# y_pred = model.predict(X_test)

# Modellperformance evaluieren
# print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# print('Coefficient of determination (R²): %.2f' % r2_score(y_test, y_pred))

# Koeffizienten des Modells anzeigen
# for idx, feature in enumerate(X.columns):
#     print(f'{feature}: {model.coef_[idx]:.4f}')

In [ ]:
# Visualisierung der Ergebnisse
# plt.figure(figsize=(10, 6))
# plt.scatter(y_test, y_pred)
# plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
# plt.xlabel('Tatsächliche Werte')
# plt.ylabel('Vorhergesagte Werte')
# plt.title('Tatsächliche vs. Vorhergesagte Werte')
# plt.tight_layout()
# plt.show()